In [5]:
%pip install google-cloud-speech
%pip install pyaudio

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
from __future__ import division  

import re 
import sys
import os
import csv
#from flask import Flask, render_template
#from app import app

from google.cloud import speech 

import pyaudio
from six.moves import queue 

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "traffic-day-86c0cf212577.json"

# Audio recording parameters
RATE = 16000
CHUNK = int(RATE / 10) #100ms 


#@app.route('/')
#def index():
    #return render_template('index.html')



class MicrophoneStream(object):
    """Opens a recording stream as a generator yielding the audio chunks."""
    
    def __init__(self, rate, chunk):
        self._rate = rate
        self._chunk = chunk
        
        # Create a thread-safe buffer of audio data
        self._buff = queue.Queue()
        self.closed = True 
    
    def __enter__(self):
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            # The API currently only supports 1-channel (momo) audio
            # https://goo.gl/z757pE
            channels=1,
            rate=self._rate,
            input=True, 
            frames_per_buffer = self._chunk,
            # Run the audio stream asynchronously to fill the buffer object
            # This is necessary so that the input device's buffer doesn't
            # overflow while the calling thread makes network requests, etc.
            
            stream_callback=self._fill_buffer,            
        )
        self.closed = False
        
        return self
    def __exit__(self, type, value, traceback):
        self._audio_stream.stop_stream()
        self._audio_stream.close()
        self.closed = True
        # Signal the generator to terminate so that the client's
        # streaming_recognize method will not block the process termination
        self._buff.put(None)
        self._audio_interface.terminate()
        
    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        """Continuously collect data from the audio stream, into the buffer."""
        self._buff.put(in_data)
        return None, pyaudio.paContinue
    
    def generator(self):
        while not self.closed:
            # Use a blocking get() to ensure there's at least one chunk of 
            # data, and stop iteration if the chunk is None, indicating the 
            # end of the audio stream
            chunk = self._buff.get()
            if chunk is None:
                return
            data = [chunk]
            
            # Now consume whatever other data's still buffered
            while True:
                try:
                    chunk = self._buff.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break
            
            yield b"".join(data)
    
def listen_print_loop(responses, writer):
    """Iterates through server responses and prints them.
        
    The responses passed is a generator that will block until a response is provided by the server.
        
    Each response may contain multiple results, and each result may contain multiple 
    alternatives. Here we print only the transcription for the top alternative of the top result.
        
    In this case, responses are provided for interim resukts as well. 
    If the response is an interim one, print a line feed at the end of it, allow the next result to overwrite it, 
    until the response is a final one. For the final one, print a newline to preserve the finalized transcription.
    """
    num_chars_printed = 0
    
    for response in responses:
        if not response.results:
            continue
            
        result = response.results[0]
        if not result.alternatives:
            continue 
            
        transcript = result.alternatives[0].transcript
        
        
        overwrite_chars = " " * (num_chars_printed - len(transcript))
        
        if not result.is_final:
            sys.stdout.write(transcript + overwrite_chars + "\r")
            sys.stdout.flush()
            
            num_chars_printed = len(transcript)
        
        else:
            print(transcript + overwrite_chars)
            
            # Exit recognition if any of the transcribed phrases could be 
            # one of our keywords.
            if re.search(r"\b(exit|quit)\b", transcript, re.I):
                print("Exiting..")
                
            
            num_chars_printed = 0
            # Write the transcript to the CSV file
            writer.writerow([transcript])
  
def main():
    language_code = "fr-FR" # a BCP-47 language tag
        
    client = speech.SpeechClient()
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code=language_code, 
    )
        
    streaming_config = speech.StreamingRecognitionConfig(
        config=config, interim_results=True 
    )
        
    with MicrophoneStream(RATE, CHUNK) as stream, open("transcript.csv", "w", newline="", encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["Transcript"])
        
        audio_generator = stream.generator()
        requests = (
            speech.StreamingRecognizeRequest(audio_content = content)
            for content in audio_generator
        ) 
            
        responses = client.streaming_recognize(streaming_config, requests)
            
        #now, put the transcription responses to uses.
        listen_print_loop(responses, writer)
        
        
        
            
if __name__ == "__main__":
    main()

ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave


Bonjour
 Bonjour
               
vous parlez de                
 quand la fête
                   
 je veux


OutOfRange: 400 Exceeded maximum allowed stream duration of 305 seconds.

In [ ]:
from google.cloud import speech
import os
import io

#setting Google credential
os.environ['GOOGLE_APPLICATION_CREDENTIALS']= 'traffic-day-86c0cf212577.json'

# create client instance 
client = speech.SpeechClient()


#the path of your audio file
file_name = "OSR_us_000_0010_8k.wav"

with io.open(file_name, "rb") as audio_file:
    content = audio_file.read()
    audio = speech.RecognitionAudio(content=content)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    enable_automatic_punctuation=True,
    audio_channel_count=1,
    language_code="en-US",
    
)

# Sends the request to google to transcribe the audio
response = client.recognize(request={"config": config, "audio": audio})
# Reads the response
for result in response.results:
    print("Transcript: {}".format(result.alternatives[0].transcript))

FileNotFoundError: [Errno 2] No such file or directory: 'OSR_us_000_0010_8k.wav'

In [ ]:
from google.cloud import speech

client = speech.SpeechClient()
speech_file = "OSR_us_000_0010_8k.wav"

with open(speech_file, "rb") as audio_file:
    content = audio_file.read()

audio = speech.RecognitionAudio(content=content)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=8000,
    language_code="en-US",
    audio_channel_count=1,
    enable_separate_recognition_per_channel=True,
)

response = client.recognize(config=config, audio=audio)

for i, result in enumerate(response.results):
    alternative = result.alternatives[0]
    print("-" * 20)
    print("First alternative of result {}".format(i))
    print("Transcript: {}".format(alternative.transcript))
    print("Channel Tag: {}".format(result.channel_tag))